## 验证实验：dixit数据看cos夹角

In [ ]:
from geneformer import InSilicoPerturber
from geneformer import InSilicoPerturberStats
from geneformer.tokenizer import TOKEN_DICTIONARY_FILE
from datasets import concatenate_datasets
import itertools as it
import logging
import numpy as np
import pickle
import re
import seaborn as sns; sns.set()
import torch
from collections import defaultdict
from datasets import Dataset, load_from_disk
from tqdm.notebook import trange
from transformers import BertForMaskedLM, BertForTokenClassification, BertForSequenceClassification
from collections import OrderedDict
from geneformer.in_silico_perturber import downsample_and_sort, \
                                         gen_attention_mask, \
                                         get_model_input_size, \
                                         load_and_filter, \
                                         load_model, \
                                         pad_tensor_list, \
                                         quant_layers, \
                                         pad_tensor, \
                                                                                                                                                                                                                                                                                                                                                                         forward_pass_single_cell, \
                                         make_perturbation_batch, \
                                         quant_cos_sims, \
                                         make_comparison_batch, \
                                         measure_length, \
                                         delete_indices, \
                                         pad_or_truncate_encoding, \
                                         remove_indices_from_emb_batch, \
                                         get_cell_state_avg_embs, \
                                         overexpress_tokens, \
                                         cos_sim_shift, \
                                         mean_nonpadding_embs

from collections import Counter
import scanpy as sc
import anndata as ad
import pandas as pd
from transformers import Trainer
from geneformer import DataCollatorForCellClassification
from sklearn.metrics import accuracy_score, f1_score
from pandas import Series,DataFrame
import random
logger = logging.getLogger(__name__)

In [2]:
# 需要修改的函数， 在geneformer代码的基础上做的修改，不然代码跑不通
def get_possible_states(cell_states_to_model):
    possible_states = []
    for key in ["start_state","goal_state"]:
        possible_states += [cell_states_to_model[key]]
    return possible_states

def random_select(input, max_sample=500, seed=2023):
    random.seed(seed)
    if len(input) <= max_sample:
        return input
    else:
        random_list = random.sample(list(np.arange(len(input))), max_sample)
        return input.select(random_list)

def cos_sim_shift_ipsc(original_emb, 
                      minibatch_emb, 
                      end_emb, 
                      perturb_group, 
                      original_minibatch_lengths = None, 
                      minibatch_lengths = None,
                      perturb_method = 'cell'):
    cos = torch.nn.CosineSimilarity(dim=2)
    
    if original_emb.size() != minibatch_emb.size():
        logger.error(
            f"Embeddings are not the same dimensions. " \
            f"original_emb is {original_emb.size()}. " \
            f"minibatch_emb is {minibatch_emb.size()}. "
        )
        raise
    
    if perturb_method == 'cell':
        if original_minibatch_lengths is not None:
            original_emb = mean_nonpadding_embs(original_emb, original_minibatch_lengths)
            end_emb = torch.unsqueeze(end_emb, 1)
            origin_v_end = cos(original_emb, end_emb)
            origin_v_end = torch.squeeze(origin_v_end)
        
        if minibatch_lengths is not None:
            perturb_emb = mean_nonpadding_embs(minibatch_emb, minibatch_lengths)
            perturb_v_end = cos(perturb_emb, end_emb)
            perturb_v_end = torch.squeeze(perturb_v_end)
    elif perturb_method == 'gene':
        origin_v_end = cos(original_emb, end_emb)
        perturb_v_end = cos(minibatch_emb, end_emb)
    
    return [(perturb_v_end - origin_v_end).to('cpu')], [((perturb_v_end - origin_v_end) / origin_v_end * 100).to('cpu')]

### 1. 变量设定
* perturb_genes = 'YY1'
* perturb_ensembl_id = ['ENSG00000100811']

In [3]:
perturb_type="delete" # perturb的类型设置为overexpress
perturb_rank_shift=None 
genes_to_perturb = ['ENSG00000100811'] # 需要perturb的gene 的ensembl_id
# random.seed(0)
# genes_to_perturb = random.sample(list(gene_token_dict.keys())[2:], 5) # random
print(genes_to_perturb)
combos=0
anchor_gene=None
model_type="diffusion_12L" # 用的什么模型要指出来，取源文件里面找
num_classes = 1
emb_mode="cell_and_gene"
cell_emb_style="mean_pool"
#filter_data={"cell_type":['fibroblast']}
filter_data=None
cell_states_to_model={'state_key': 'condition',
                      'start_state': 'control', 
                      'goal_state': 'YY1'} #是什么状态要说明
max_ncells=2000
#cell_inds_to_perturb={'start':0, 'end':500}
cell_inds_to_perturb='all'
emb_layer=-1
forward_batch_size=100 #batchsize要好好设定，不能超出内存
nproc=16
token_dictionary_file=TOKEN_DICTIONARY_FILE

if genes_to_perturb == "all":
    perturb_group = False
else:
    perturb_group = True
    if (anchor_gene != None) or (combos != 0):
        anchor_gene = None
        combos = 0
        logger.warning( 
            "anchor_gene set to None and combos set to 0. " \
            "If providing list of genes to perturb, " \
            "list of genes_to_perturb will be perturbed together, "\
            "without anchor gene or combinations.")
        
# load token dictionary (Ensembl IDs:token)
with open(token_dictionary_file, "rb") as f:
    gene_token_dict = pickle.load(f)
pad_token_id = gene_token_dict.get("<pad>")

if anchor_gene is None:
    anchor_token = None
else:
    try:
        anchor_token = [gene_token_dict[anchor_gene]]
    except KeyError:
        logger.error(
            f"Anchor gene {anchor_gene} not in token dictionary."
        )
        raise

if genes_to_perturb == "all":
    tokens_to_perturb = "all"
else:
    missing_genes = [gene for gene in genes_to_perturb if gene not in gene_token_dict.keys()]
    if len(missing_genes) == len(genes_to_perturb):
        logger.error(
            "None of the provided genes to perturb are in token dictionary."
        )
        raise
    elif len(missing_genes)>0:
        logger.warning(
            f"Genes to perturb {missing_genes} are not in token dictionary.")
    tokens_to_perturb = [gene_token_dict.get(gene) for gene in genes_to_perturb]

['ENSG00000100811']


### 2. perturb-data

In [4]:
model_directory = '/mnt/nfs/xz/diffusion/db/Diffusion-BERT/model_out/230901_025119_model_name_gene_lr_5e-06_seed_42_numsteps_128_sample_Categorical_schedule_mutual_hybridlambda_0.01_wordfreqlambda_0.3_fromscratch_False_timestep_none_ckpts/best_0_14999.th' #diffusion12L
model = load_model(model_type, num_classes, model_directory)
layer_to_quant = quant_layers(model)+emb_layer #取哪一层的embedding

data_path = '/mnt/nfs/wbzhang/data/perturber/experiment/data/dixit/tokenized_dixit.dataset'
raw_data = load_from_disk(data_path)
control_df = raw_data.filter(lambda x: x['condition'] == 'control')
stimulated_df = raw_data.filter(lambda x: x['condition'] == 'YY1')
print('The count of control:', len(control_df))
print('The count of stimulated:', len(stimulated_df))

filtered_input_data = concatenate_datasets([random_select(control_df), random_select(stimulated_df)])
filtered_input_data

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at /mnt/nfs/xz/geneformer/Geneformer34m/examples/pretraining_new_model/output_own/models/230813_170132_geneformer_30M_L12_emb512_SL2048_E3_B12_LR0.0005_LSlinear_WU10000_Oadamw_DS64/models and are newly initialized: ['classifier.bias', 'bert.pooler.dense.bias', 'classifier.weight', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The count of control: 4475
The count of stimulated: 1025


Dataset({
    features: ['input_ids', 'condition', 'length'],
    num_rows: 1000
})

In [5]:
# cell_states_to_model is Not None
state_name = cell_states_to_model["state_key"]
state_values = filtered_input_data[state_name] # 所有state的值
for value in get_possible_states(cell_states_to_model):
    if value not in state_values:
        logger.error(
            f"{value} is not present in the dataset's {state_name} attribute.")
        raise
downsampled_data = downsample_and_sort(filtered_input_data, max_ncells)

# 拿来做对比的embedding
state_embs_dict = get_cell_state_avg_embs(model,
                                          downsampled_data,
                                          cell_states_to_model,
                                          layer_to_quant,
                                          pad_token_id,
                                          forward_batch_size,
                                          nproc)
cos1 = torch.nn.CosineSimilarity(dim=1)
print('The cos between control and YY1:', cos1(state_embs_dict['control'], state_embs_dict['YY1']))

# filter for start state cells
start_state = cell_states_to_model["start_state"] # 从0状态开始
def filter_for_origin(example):
    return example[state_name] in [start_state]
filtered_input_data = filtered_input_data.filter(filter_for_origin, num_proc=nproc) #只选择全部数据中对应start_state的数据

#output_path_prefix = f"{output_directory}in_silico_{perturb_type}_{output_prefix}_dict_1Kbatch"
model_input_size = get_model_input_size(model)
cos_sims_dict = defaultdict(list)
pickle_batch = -1
filtered_input_data = downsample_and_sort(filtered_input_data, max_ncells)
print('The size of filtered_input_data:', len(filtered_input_data))

The cos between control and YY1: tensor([0.9996], device='cuda:0')
The size of filtered_input_data: 500


### 3. In_silico_perturb

In [6]:
# 'delete'
min_genes = len(tokens_to_perturb)
def if_has_tokens_to_perturb(example):
    return (len(set(example["input_ids"]).intersection(tokens_to_perturb))>=min_genes)
filtered_input_data = filtered_input_data.filter(if_has_tokens_to_perturb, num_proc=nproc)
if len(filtered_input_data) == 0:
    logger.error(
            "No cells in dataset contain all genes to perturb as a group.")
    raise
filtered_input_data

Filter (num_proc=16):   0%|          | 0/500 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'condition', 'length'],
    num_rows: 297
})

In [7]:
#选择cell的index
if cell_inds_to_perturb != "all":
    if cell_inds_to_perturb["start"] >= len(filtered_input_data):
        logger.error("cell_inds_to_perturb['start'] is larger than the filtered dataset.")
        raise
    if cell_inds_to_perturb["end"] > len(filtered_input_data):
        logger.warning("cell_inds_to_perturb['end'] is larger than the filtered dataset. \
                       Setting to the end of the filtered dataset.")
        cell_inds_to_perturb["end"] = len(filtered_input_data)
    filtered_input_data = filtered_input_data.select([i for i in range(cell_inds_to_perturb["start"], cell_inds_to_perturb["end"])])
#cell_inds_to_perturb == "all"
# perturb_group == True
def make_group_perturbation_batch(example):
    example_input_ids = example["input_ids"]
    example["tokens_to_perturb"] = tokens_to_perturb
    indices_to_perturb = [example_input_ids.index(token) if token in example_input_ids else None for token in tokens_to_perturb]
    indices_to_perturb = [item for item in indices_to_perturb if item is not None]
    if len(indices_to_perturb) > 0:
        example["perturb_index"] = indices_to_perturb
    else:
        # -100 indicates tokens to overexpress are not present in rank value encoding
        example["perturb_index"] = [-100] #如果没有出现就是-100
    if perturb_type == "delete":
        example = delete_indices(example)
    elif perturb_type == "overexpress":
        example = overexpress_tokens(example) 
    return example
    
perturbation_batch = filtered_input_data.map(make_group_perturbation_batch, num_proc=nproc) # 如果gene在里面，就往前调，如果没有，就直接加在前面
indices_to_perturb = perturbation_batch["perturb_index"]

Map (num_proc=16):   0%|          | 0/297 [00:00<?, ? examples/s]

### 4. quant_cos_sims

In [8]:
original_emb = filtered_input_data #只有这一个参数需要去特别指定 #原始的perturb矩阵
cos = torch.nn.CosineSimilarity(dim=2)
total_batch_length = len(perturbation_batch)
if ((total_batch_length-1)/forward_batch_size).is_integer():
    forward_batch_size = forward_batch_size-1
# cell_states_to_model is Not None
possible_states = get_possible_states(cell_states_to_model) #可能的所有状态
cos_sims_vs_alt_dict = dict(zip(possible_states,[[] for i in range(len(possible_states))]))
#百分比修改
cos_sims_vs_alt_dict_percent = dict(zip(possible_states,[[] for i in range(len(possible_states))]))
perturbation_batch = perturbation_batch.map(
        measure_length, num_proc=nproc
    ) #对perturb的input_ids重新计算length

#开始取perturb embedding
def compute_batch_embeddings(minibatch, _max_len = None):
    minibatch_lengths = minibatch["length"]
    minibatch_length_set = set(minibatch_lengths)
    max_len = model_input_size

    if (len(minibatch_length_set) > 1) or (max(minibatch_length_set) > max_len):
        needs_pad_or_trunc = True
    else:
        needs_pad_or_trunc = False
        max_len = max(minibatch_length_set)


    if needs_pad_or_trunc == True:
        if _max_len is None:
            max_len = min(max(minibatch_length_set), max_len)
        else:
            max_len = _max_len
        def pad_or_trunc_example(example):
            example["input_ids"] = pad_or_truncate_encoding(example["input_ids"], 
                                                           pad_token_id, 
                                                           max_len)
            return example
        minibatch = minibatch.map(pad_or_trunc_example, num_proc=nproc)
        
    minibatch.set_format(type="torch")
    
    input_data_minibatch = minibatch["input_ids"]
    attention_mask = gen_attention_mask(minibatch, max_len)
    
    # extract embeddings for perturbation minibatch
    with torch.no_grad():
        outputs = model(
            input_ids = input_data_minibatch.to("cuda"),
            attention_mask = attention_mask
        )

    return outputs, max_len

Map (num_proc=16):   0%|          | 0/297 [00:00<?, ? examples/s]

In [9]:
for i in range(0, total_batch_length, forward_batch_size):
    max_range = min(i+forward_batch_size, total_batch_length)

    perturbation_minibatch = perturbation_batch.select([i for i in range(i, max_range)])
    outputs, mini_max_len = compute_batch_embeddings(perturbation_minibatch)

    if len(indices_to_perturb)>1:
        minibatch_emb = torch.squeeze(outputs.hidden_states[layer_to_quant])
    else:
        minibatch_emb = outputs.hidden_states[layer_to_quant]
    
    # perturb_group == True:
    original_minibatch = original_emb.select([i for i in range(i, max_range)])
    original_outputs, orig_max_len = compute_batch_embeddings(original_minibatch)
    if len(indices_to_perturb)>1:
        original_minibatch_emb = torch.squeeze(original_outputs.hidden_states[layer_to_quant])
    else:
        original_minibatch_emb = original_outputs.hidden_states[layer_to_quant]

    if len(tokens_to_perturb) == 1:
        indices_to_perturb_minibatch = [idx if idx != [-100] else [orig_max_len - 1]
                                        for idx in indices_to_perturb[i:max_range]]
    else:
        num_perturbed = len(tokens_to_perturb)
        indices_to_perturb_minibatch = []
        end_range = [i for i in range(orig_max_len - len(tokens_to_perturb), orig_max_len)]
        for idx in indices_to_perturb[i:max_range]:
            if idx == [-100]:
                indices_to_perturb_minibatch.append(end_range)
            elif len(idx) < len(tokens_to_perturb):
                # 代码不对 有修改
                indices_to_perturb_minibatch.append(idx + end_range[-num_perturbed+len(idx):])
            else:
                indices_to_perturb_minibatch.append(idx)
    
    original_minibatch_emb = remove_indices_from_emb_batch(original_minibatch_emb, 
                                                           indices_to_perturb_minibatch, 
                                                           gene_dim=1)

    original_minibatch_lengths = torch.tensor(original_minibatch["length"], device="cuda")
    minibatch_lengths = torch.tensor(perturbation_minibatch["length"], device="cuda")
    print(original_minibatch_emb.shape)
    print(minibatch_emb.shape)
    for state in possible_states:
        '''
        cos_sim_shift:有大量修改，不取mean，每个cell 的每个gene算cos，先整合起来
        '''
        cos_sims_vs_alt_dict[state] += cos_sim_shift_ipsc(original_minibatch_emb, 
                                                    minibatch_emb, 
                                                    state_embs_dict[state],
                                                    perturb_group,
                                                    torch.tensor(original_minibatch_lengths, device="cuda"),
                                                    torch.tensor(minibatch_lengths, device="cuda"),
                                                    perturb_method='cell')[0]
        cos_sims_vs_alt_dict_percent[state] += cos_sim_shift_ipsc(original_minibatch_emb, 
                                                    minibatch_emb, 
                                                    state_embs_dict[state],
                                                    perturb_group,
                                                    torch.tensor(original_minibatch_lengths, device="cuda"),
                                                    torch.tensor(minibatch_lengths, device="cuda"),
                                                    perturb_method='cell')[1]
        
    del outputs
    del minibatch_emb
    del mini_max_len
    del original_minibatch
    del original_outputs
    del orig_max_len
    del original_minibatch_emb
    del original_minibatch_lengths
    del perturbation_minibatch
    del minibatch_lengths
# cell_states_to_model is not None
for state in possible_states:
    cos_sims_vs_alt_dict[state] = torch.cat(cos_sims_vs_alt_dict[state])
    cos_sims_vs_alt_dict_percent[state] = torch.cat(cos_sims_vs_alt_dict_percent[state])

/tmp/ipykernel_61835/2221873459.py:40: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  original_minibatch_lengths = torch.tensor(original_minibatch["length"], device="cuda")
/tmp/ipykernel_61835/2221873459.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  minibatch_lengths = torch.tensor(perturbation_minibatch["length"], device="cuda")
/tmp/ipykernel_61835/2221873459.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(original_minibatch_lengths, device="cuda"),
/tmp/ipykernel_61835/2221873459.py:53: UserWarning: To copy con

torch.Size([100, 2047, 512])
torch.Size([100, 2047, 512])
torch.Size([100, 2047, 512])
torch.Size([100, 2047, 512])


Map (num_proc=16):   0%|          | 0/97 [00:00<?, ? examples/s]

Map (num_proc=16):   0%|          | 0/97 [00:00<?, ? examples/s]

torch.Size([97, 2047, 512])
torch.Size([97, 2047, 512])


In [11]:
print('cos shift:', torch.mean(cos_sims_vs_alt_dict['YY1']))

cos shift: tensor(-1.0724e-05)
